<a href="https://colab.research.google.com/github/bilpang28/Aaron/blob/main/tugas_akhir_nabila_pangastuti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Program Aplikasi Rekonstruksi Orang Hilang

Aplikasi ini dirancang dengan menggunakan react native sebagai frontend dan django sebagai backend dengan tujuan untuk membantu pencarian orang hilang melalui keilmuwan artificial intelligent atau lebih berfokus pada model implementasi fisherface.

# A. persiapan google colab

Langkah pertama yang harus dilakukan agar berjalan dengan baik, sebagai berikut.

1. buat notebook baru pada google colab
2. berikan judul pada notebook baru
3. aktifkan google drive untuk menyimpan file yang diperlukan
4. pilih folder penyimpanan di drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/FaceReconstruction

/content/drive/MyDrive/FaceReconstruction


In [3]:
!ls /content/drive/MyDrive/FaceReconstruction

backend  BioID-FaceDatabase-V1.2  CFD  frontend  KinFace_V2


# B. instalasi django

Selanjutnya instalasi django sebagai backend dari aplikasi rekonstruksi wajah

1. instal django dan library pendukung
2. buat project backend django
3. masuk kedalam folder backend

In [4]:
!pip install django djangorestframework

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.9 MB/s eta 0:00:00


In [5]:
!django-admin startproject backend

CommandError: '/content/drive/MyDrive/FaceReconstruction/backend' already exists


In [6]:
%cd /content/drive/MyDrive/FaceReconstruction/backend

/content/drive/MyDrive/FaceReconstruction/backend


# C. konfigurasi backend django

Langkah ketiga merupakan konfigurasi backend django sebagai berikut.

1. tambahkan aplikasi untuk API
2. daftarkan API di file settings.py
3. buat endpoint API menggunakan django rest framework
    * buat serializer di api/serializers.py
    * tambahkan model di api/models.py
    * tambahkan view di api/views.py
    * buat url routing di api/urls.py
    * sambungkan backend/urls.py
    * tambahkan depedencies
    * masukkan dalam installed apps
    'api',
    'rest_framework',
    'corsheaders'

4. tambahkan dalam settings.py
    MEDIA_URL = '/media/',
    MEDIA_ROOT = BASE_DIR / 'media',
    AUTH_USER_MODEL = 'api.User',

5. buat migrasinya
6. migrasikan dan tampilkan hasil migrasinya
7. jalankan aplikasi backend nya



In [7]:
!python manage.py startapp api

CommandError: 'api' conflicts with the name of an existing Python module and cannot be used as an app name. Please try another name.


In [8]:
import os

folder_name = "api"
os.makedirs(folder_name, exist_ok=True)
file_names = ["models.py", "serializers.py", "views.py", "urls.py", "service.py"]
file_contents = [
'''from django.db import models
from django.contrib.auth.models import AbstractUser

class User(AbstractUser):
      email = models.EmailField(unique=True)

class MissingPersonReport(models.Model):
      reporter = models.ForeignKey(User, on_delete=models.CASCADE)
      name = models.CharField(max_length=255)
      age = models.PositiveIntegerField()
      gender = models.CharField(max_length=10, choices=[('Male', 'Male'), ('Female', 'Female')])
      description = models.TextField()
      last_seen_location = models.CharField(max_length=255)
      last_seen_date = models.DateField()
      photo = models.ImageField(upload_to='missing_person_photos/')
      created_at = models.DateTimeField(auto_now_add=True)

class Notification(models.Model):
      user = models.ForeignKey(User, on_delete=models.CASCADE)
      message = models.TextField()
      is_read = models.BooleanField(default=False)
      created_at = models.DateTimeField(auto_now_add=True)
      ''',
'''from rest_framework import serializers
from .models import User, MissingPersonReport, Notification

class UserSerializer(serializers.ModelSerializer):
      class Meta:
            model = User
            fields = ['id', 'username', 'email', 'first_name', 'last_name']

class MissingPersonReportSerializer(serializers.ModelSerializer):
      class Meta:
            model = MissingPersonReport
            fields = '__all__'

class NotificationSerializer(serializers.ModelSerializer):
      class Meta:
            model = Notification
            fields = '__all__'
      ''',
'''from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework.permissions import IsAuthenticated
from rest_framework import status, viewsets
from django.contrib.auth import authenticate, login, logout
from .models import MissingPersonReport, Notification
from .serializers import MissingPersonReportSerializer, NotificationSerializer, UserSerializer
from django.http import JsonResponse
from .services import fisherface, simulate_aging
import cv2
import numpy as np

class RegisterView(APIView):
      def post(self, request):
          serializer = UserSerializer(data=request.data)
          if serializer.is_valid():
             serializer.save()
             return Response(serializer.data, status=status.HTTP_201_CREATED)
          return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

class LoginView(APIView):
      def post(self, request):
          username = request.data.get('username')
          password = request.data.get('password')
          user = authenticate(username=username, password=password)
          if user:
             login(request, user)
             return Response({"message": "Login successful"}, status=status.HTTP_200_OK)
          return Response({"error": "Invalid credentials"}, status=status.HTTP_400_BAD_REQUEST)

class LogoutView(APIView):
      permission_classes = [IsAuthenticated]

      def post(self, request):
          logout(request)
          return Response({"message": "Logged out successfully"}, status=status.HTTP_200_OK)

class MissingPersonReportViewSet(viewsets.ModelViewSet):
          queryset = MissingPersonReport.objects.all()
          serializer_class = MissingPersonReportSerializer
          permission_classes = [IsAuthenticated]

class NotificationViewSet(viewsets.ModelViewSet):
          queryset = Notification.objects.all()
          serializer_class = NotificationSerializer
          permission_classes = [IsAuthenticated]

      def reconstruct_face(request):
          if request.method == "POST":
              images = request.FILES.getlist("images")
              labels = request.POST.getlist("labels")

              data = []
              for image in images:
                  img = cv2.imdecode(np.frombuffer(image.read(), np.uint8), cv2.IMREAD_COLOR)
                  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                  data.append(gray.flatten())

              data = np.array(data)
              labels = np.array(labels, dtype=int)
              pca_mean, pca_eigenvectors, lda_scalings = fisherface(data, labels)
              aged_faces = simulate_aging(lda_scalings)
              return JsonResponse({"aged_faces": [face.tolist() for face in aged_faces]})

          return JsonResponse({"error": "Invalid request method"}, status=400)
      ''',
'''from django.urls import path, include
from rest_framework.routers import DefaultRouter
from .views import RegisterView, LoginView, LogoutView, MissingPersonReportViewSet, NotificationViewSet, reconstruct_face

      router = DefaultRouter()
      router.register(r'reports', MissingPersonReportViewSet, basename='report')
      router.register(r'notifications', NotificationViewSet, basename='notification')

      urlpatterns = [
          path('auth/register/', RegisterView.as_view(), name='register'),
          path('auth/login/', LoginView.as_view(), name='login'),
          path('auth/logout/', LogoutView.as_view(), name='logout'),
          path('reconstruct/', reconstruct_face, name='reconstruct_face'),
          path('', include(router.urls)),
      ]
''',
'''import numpy as np
import cv2
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

      def calculate_intensity_histogram(image):
          """Menghitung histogram intensitas."""
          gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
          histogram = cv2.calcHist([gray], [0], None, [256], [0, 256])
          return histogram

      def pca_transformation(data, n_components=5):
          """Menghitung PCA."""
          mean = np.mean(data, axis=0)
          centered_data = data - mean
          cov_matrix = np.cov(centered_data, rowvar=False)
          eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
          idx = eigenvalues.argsort()[::-1]
          eigenvectors = eigenvectors[:, idx]
          eigenvalues = eigenvalues[idx]
          return mean, eigenvalues[:n_components], eigenvectors[:, :n_components]

      def lda_transformation(data, labels):
          """Menghitung LDA."""
          lda = LDA(n_components=len(np.unique(labels)) - 1)
          transformed_data = lda.fit_transform(data, labels)
          return lda.means_, lda.scalings_

      def fisherface(data, labels, n_components=5):
          """Menggabungkan PCA dan LDA."""
          pca_mean, pca_eigenvalues, pca_eigenvectors = pca_transformation(data, n_components=n_components)
          transformed_data = np.dot(data - pca_mean, pca_eigenvectors)
          lda_means, lda_scalings = lda_transformation(transformed_data, labels)
          return pca_mean, pca_eigenvectors, lda_scalings

      def simulate_aging(fisherfaces, n_periods=4):
          """Mensimulasikan penuaan wajah."""
          aged_faces = []
          for i in range(1, n_periods + 1):
              aging_factor = 1 + (i * 0.1)
              aged_face = fisherfaces * aging_factor
              aged_faces.append(aged_face)
          return aged_faces
      ''',
]

for file_name, content in zip(file_names, file_contents):
    file_path = os.path.join(folder_name, file_name)
    with open(file_path, "w") as file:
        file.write(content)

print(f"{len(file_names)} berhasil '{folder_name}'.")


5 berhasil 'api'.


In [9]:
!pip install django-cors-headers

In [10]:
!python manage.py makemigrations
!python manage.py showmigrations
!python manage.py migrate
!python manage.py runserver

No changes detected
admin
 [X] 0001_initial
 [X] 0002_logentry_remove_auto_add
 [X] 0003_logentry_add_action_flag_choices
auth
 [X] 0001_initial
 [X] 0002_alter_permission_name_max_length
 [X] 0003_alter_user_email_max_length
 [X] 0004_alter_user_username_opts
 [X] 0005_alter_user_last_login_null
 [X] 0006_require_contenttypes_0002
 [X] 0007_alter_validators_add_error_messages
 [X] 0008_alter_user_username_max_length
 [X] 0009_alter_user_last_name_max_length
 [X] 0010_alter_group_name_max_length
 [X] 0011_update_proxy_permissions
 [X] 0012_alter_user_first_name_max_length
contenttypes
 [X] 0001_initial
 [X] 0002_remove_content_type_name
sessions
 [X] 0001_initial
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  No migrations to apply.
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
January 06, 2025 - 12:58:24
Django version 5.1.4, using settings 'backend.sett

# D. setup react native

Melakukan langkah awal instalasi setup react native sebagai frontend aplikasi

1. kembali ke folder awal yaitu FaceReconstrucion
2. install node js
3. buat project baru dengan nama frontend
4. install react native dan buat project frontend
5. masuk kedalam folder frontend
6. menginstal kepada depedencies
7. menghapus folder yang tidak dibutuhkan
8. membuat 2 folder baru folder src dan assets serta file api.js
9. membuat folder baru di dalam folder src dan masuk ke path folder src
10. membuat file baru di dalam folder screen dan memasukkan kode sesuai dengan halaman masing - masing
11. membuat file baru di dalam folder navigations dan memasukkan kode sesuai dengan halaman masing - masing
12. membuat file baru di dalam folder components dan memasukkan kode sesuai dengan halaman masing - masing
13. membuat folder image dan font pada folder assets
14. memasukkan semua assets
15. menjalankan program aplikasi react native


In [11]:
%cd /content/drive/MyDrive/FaceReconstruction

/content/drive/MyDrive/FaceReconstruction


In [12]:
!curl -sL https://deb.nodesource.com/setup_22.x | sudo -E bash -
!sudo apt-get install -y nodejs

2025-01-06 13:20:37 - Installing pre-requisites
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,197 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,632 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Pack

In [13]:
!mkdir frontend
%cd frontend
!npm init -y

mkdir: cannot create directory ‘frontend’: File exists
/content/drive/MyDrive/FaceReconstruction/frontend
⠙⠹⠸⠼⠴⠦⠧⠇Wrote to /content/drive/MyDrive/FaceReconstruction/frontend/package.json:

{
  "name": "frontend",
  "version": "1.0.0",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC",
  "description": ""
}



⠙

In [14]:
!npm install -g react-native-cli

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸npm warn deprecated rimraf@2.7.1: Rimraf versions prior to v4 are no longer supported
⠸npm warn deprecated inflight@1.0.6: This module is not supported, and leaks memory. Do not use it. Check out lru-cache if you want a good and tested way to coalesce async requests by a key value, which is much more comprehensive and powerful.
⠸npm warn deprecated glob@7.2.3: Glob versions prior to v9 are no longer supported
⠼⠴
added 98 packages in 10s
⠴
⠴45 packages are looking for funding
⠴  run `npm fund` for details
⠴

In [15]:
!npx create-expo-app@latest

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋Need to install the following packages:
create-expo-app@3.2.0
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧Creating an Expo project using the default template.

To choose from all available templates pass in the --template arg:
  $ npx create-expo-app --template

To choose from all available examples pass in the --example arg:
  $ npx create-expo-app --example

? What is your app named? › my-app78? What is your app named? › f78? What is your app named? › fr78? What is your app named? › fro78? What is your app named? › fron78? What is your app named? › front78? What is your app named? › fronte78? What is your app named? › fronten78? What is your app named? › frontend78✔ What is your app named? … frontend78
The directory frontend has files that might be overwritten:

  README.md
  app
  app.json
  assets
  components
  constants
  hooks
  node_modules
  package.json
  scripts
  tsconfig.json

Try using a new directory name, or moving these files.

⠙

In [16]:
%cd /content/drive/MyDrive/FaceReconstruction/frontend/frontend

/content/drive/MyDrive/FaceReconstruction/frontend/frontend


In [17]:
!npm install axios

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋

In [18]:
import os
import shutil

folders_to_manage = [
    "/content/drive/MyDrive/FaceReconstruction/frontend/frontend/app",
    "/content/drive/MyDrive/FaceReconstruction/frontend/frontend/assets",
    "/content/drive/MyDrive/FaceReconstruction/frontend/frontend/components",
    "/content/drive/MyDrive/FaceReconstruction/frontend/frontend/constants",
    "/content/drive/MyDrive/FaceReconstruction/frontend/frontend/hooks",
    "/content/drive/MyDrive/FaceReconstruction/frontend/frontend/scripts",
]

def delete_folders(folder_list):
    for folder in folder_list:
        try:
            shutil.rmtree(folder, ignore_errors=True)
            print(f"Folder deleted: {folder}")
        except Exception as e:
            print(f"Error deleting folder {folder}: {e}")

print("\nDeleting Folders")
delete_folders(folders_to_manage)



Deleting Folders
Folder deleted: /content/drive/MyDrive/FaceReconstruction/frontend/frontend/app
Folder deleted: /content/drive/MyDrive/FaceReconstruction/frontend/frontend/assets
Folder deleted: /content/drive/MyDrive/FaceReconstruction/frontend/frontend/components
Folder deleted: /content/drive/MyDrive/FaceReconstruction/frontend/frontend/constants
Folder deleted: /content/drive/MyDrive/FaceReconstruction/frontend/frontend/hooks
Folder deleted: /content/drive/MyDrive/FaceReconstruction/frontend/frontend/scripts


In [19]:
import os
import shutil

folders_to_manage = [
    "/content/drive/MyDrive/FaceReconstruction/frontend/frontend/src",
    "/content/drive/MyDrive/FaceReconstruction/frontend/frontend/assets",
]

def create_folders(folder_list):
    for folder in folder_list:
        try:
            os.makedirs(folder, exist_ok=True)
            print(f"Folder created: {folder}")
        except Exception as e:
            print(f"Error creating folder {folder}: {e}")

print("Creating Folders")
create_folders(folders_to_manage)

Creating Folders
Folder created: /content/drive/MyDrive/FaceReconstruction/frontend/frontend/src
Folder created: /content/drive/MyDrive/FaceReconstruction/frontend/frontend/assets


In [20]:
import os
import shutil

folders_to_manage = [
    "/content/drive/MyDrive/FaceReconstruction/frontend/frontend/src/components",
    "/content/drive/MyDrive/FaceReconstruction/frontend/frontend/src/navigations",
    "/content/drive/MyDrive/FaceReconstruction/frontend/frontend/src/screens"
]

def create_folders(folder_list):
    for folder in folder_list:
        try:
            os.makedirs(folder, exist_ok=True)
            print(f"Folder created: {folder}")
        except Exception as e:
            print(f"Error creating folder {folder}: {e}")

print("Creating Folders")
create_folders(folders_to_manage)

Creating Folders
Folder created: /content/drive/MyDrive/FaceReconstruction/frontend/frontend/src/components
Folder created: /content/drive/MyDrive/FaceReconstruction/frontend/frontend/src/navigations
Folder created: /content/drive/MyDrive/FaceReconstruction/frontend/frontend/src/screens


In [21]:
import os

folder_name = "api"
os.makedirs(folder_name, exist_ok=True)
file_names = ["api.js"]
file_contents = [
    ''' import axios from 'axios';

const api = axios.create({
  baseURL: 'https://127.0.0.1/api',
  headers: {
    'Content-Type': 'application/json',
  },
});

export default api;
'''
]

for file_name, content in zip(file_names, file_contents):
    file_path = os.path.join(folder_name, file_name)
    with open(file_path, "w") as file:
        file.write(content)

print(f"{len(file_names)} berhasil '{folder_name}'.")


1 berhasil 'api'.


In [22]:
%cd /content/drive/MyDrive/FaceReconstruction/frontend/frontend/src

/content/drive/MyDrive/FaceReconstruction/frontend/frontend/src


In [23]:
import os

folder_name = "components"
os.makedirs(folder_name, exist_ok=True)
file_names = ["Button.js", "Header.js", "TabSwitcher.js", "FormInput.js"]
file_contents = [
    '''import React from 'react';
import { TouchableOpacity, Text, StyleSheet } from 'react-native';

const Button = ({ title, onPress }) => {
  return (
    <TouchableOpacity style={styles.button} onPress={onPress}>
      <Text style={styles.buttonText}>{title}</Text>
    </TouchableOpacity>
  );
};

const styles = StyleSheet.create({
  button: {
    backgroundColor: '#000080',
    paddingVertical: 15,
    borderRadius: 8,
    alignItems: 'center',
  },
  buttonText: {
    color: '#FFF',
    fontSize: 16,
    fontWeight: 'bold',
  },
});

export default Button;
''',
    '''import React from 'react';
import { View, Text, StyleSheet } from 'react-native';

const Header = ({ title }) => {
  return (
    <View style={styles.headerContainer}>
      <Text style={styles.headerText}>{title}</Text>
    </View>
  );
};

const styles = StyleSheet.create({
  headerContainer: {
    alignItems: 'center',
    marginBottom: 20,
  },
  headerText: {
    fontSize: 24,
    fontWeight: 'bold',
    color: '#000',
  },
});

export default Header;''',
    '''import React from 'react';
import { View, Text, TouchableOpacity, StyleSheet } from 'react-native';

const TabSwitcher = ({ activeTab }) => {
  return (
    <View style={styles.tabContainer}>
      <TouchableOpacity style={[styles.tab, activeTab === 'Login' && styles.activeTab]}>
        <Text style={activeTab === 'Login' ? styles.activeTabText : styles.tabText}>Login</Text>
      </TouchableOpacity>
      <TouchableOpacity style={[styles.tab, activeTab === 'Register' && styles.activeTab]}>
        <Text style={activeTab === 'Register' ? styles.activeTabText : styles.tabText}>Register</Text>
      </TouchableOpacity>
    </View>
  );
};

const styles = StyleSheet.create({
  tabContainer: {
    flexDirection: 'row',
    justifyContent: 'center',
    marginBottom: 20,
  },
  tab: {
    paddingVertical: 10,
    paddingHorizontal: 20,
    borderRadius: 5,
  },
  activeTab: {
    backgroundColor: '#FFA500',
  },
  tabText: {
    fontSize: 16,
    color: '#000',
  },
  activeTabText: {
    fontSize: 16,
    color: '#FFF',
    fontWeight: 'bold',
  },
});

export default TabSwitcher;
''',
    '''import React from 'react';
import { TextInput, StyleSheet } from 'react-native';

const FormInput = ({ placeholder, value, onChangeText, keyboardType, secureTextEntry }) => {
  return (
    <TextInput
      style={styles.input}
      placeholder={placeholder}
      value={value}
      onChangeText={onChangeText}
      keyboardType={keyboardType}
      secureTextEntry={secureTextEntry}
    />
  );
};

const styles = StyleSheet.create({
  input: {
    height: 50,
    borderColor: '#CCC',
    borderWidth: 1,
    borderRadius: 8,
    paddingHorizontal: 10,
    marginBottom: 15,
    backgroundColor: '#FFF',
  },
});

export default FormInput;'''
]

for file_name, content in zip(file_names, file_contents):
    file_path = os.path.join(folder_name, file_name)
    with open(file_path, "w") as file:
        file.write(content)

print(f"{len(file_names)} berhasil '{folder_name}'.")


4 berhasil 'components'.


In [24]:
import os

folder_name = "navigations"
os.makedirs(folder_name, exist_ok=True)
file_names = ["AppNavigators.js"]
file_contents = [
    ''''import React from 'react';
import { NavigationContainer } from '@react-navigation/native';
import { createStackNavigator } from '@react-navigation/stack';

import LoginScreen from './LoginScreen';
import RegisterScreen from './RegisterScreen';
import HomeScreen from './HomeScreen';
import FaceScreen from './FaceScreen';
import ProfileScreen from './ProfileScreen';
import ArticleScreen from './ArticleScreen';
import NotificationScreen from './NotificationScreen';
import GraphicScreen from './GraphicScreen';
import SearchScreen from './SearchScreen';
import LandingScreen from './LandingScreen';
import VerificationEmailScreen from './VerificationEmailScreen';
import ForgotPassword from './ForgotPassword';
import LaporanPelaporan from './LaporanPelaporan';

const Stack = createStackNavigator();

const AppNavigator = () => {
  return (
    <NavigationContainer>
      <Stack.Navigator initialRouteName="LandingScreen">
        <Stack.Screen name="LandingScreen" component={LandingScreen} options={{ headerShown: false }} />
        <Stack.Screen name="LoginScreen" component={LoginScreen} options={{ title: 'Login' }} />
        <Stack.Screen name="RegisterScreen" component={RegisterScreen} options={{ title: 'Register' }} />
        <Stack.Screen name="HomeScreen" component={HomeScreen} options={{ title: 'Home' }} />
        <Stack.Screen name="FaceScreen" component={FaceScreen} options={{ title: 'Face Recognition' }} />
        <Stack.Screen name="ProfileScreen" component={ProfileScreen} options={{ title: 'Profile' }} />
        <Stack.Screen name="ArticleScreen" component={ArticleScreen} options={{ title: 'Articles' }} />
        <Stack.Screen name="NotificationScreen" component={NotificationScreen} options={{ title: 'Notifications' }} />
        <Stack.Screen name="GraphicScreen" component={GraphicScreen} options={{ title: 'Graphics' }} />
        <Stack.Screen name="SearchScreen" component={SearchScreen} options={{ title: 'Search' }} />
        <Stack.Screen name="VerificationEmailScreen" component={VerificationEmailScreen} options={{ title: 'Verify Email' }} />
        <Stack.Screen name="ForgotPassword" component={ForgotPassword} options={{ title: 'Forgot Password' }} />
        <Stack.Screen name="LaporanPelaporan" component={LaporanPelaporan} options={{ title: 'Reports' }} />
      </Stack.Navigator>
    </NavigationContainer>
  );
};

export default AppNavigator;'''

]

for file_name, content in zip(file_names, file_contents):
    file_path = os.path.join(folder_name, file_name)
    with open(file_path, "w") as file:
        file.write(content)

print(f"{len(file_names)} berhasil '{folder_name}'.")


1 berhasil 'navigations'.


In [25]:
import os

folder_name = "screens"
os.makedirs(folder_name, exist_ok=True)
file_names = ["LoginScreen.js", "RegisterScreen.js", "HomeScreen.js", "FaceScreen.js", "ProfileScreen.js", "ArticleScreen.js", "Notificationscreen.js", "GraphicScreen.js", "SearchScreen.js", "Landingscreen.js", "VerificationEmailScreen.js", "ForgotPassword.js", "LaporanPelaporan.js"]
file_contents = [
    ''' import React, { useState } from 'react';
import { View, Text, TextInput, Button, StyleSheet } from 'react-native';
import api from './api';

const LoginScreen = ({ navigation }) => {
  const [email, setEmail] = useState('');
  const [password, setPassword] = useState('');

  const handleLogin = async () => {
    try {
      const response = await api.post('/auth/login/', { email, password });
      alert('Login successful!');
      navigation.navigate('Home');
    } catch (error) {
      console.error(error);
      alert('Login failed. Please check your credentials.');
    }
  };

  return (
    <View style={styles.container}>
      <Text style={styles.title}>Login</Text>
      <TextInput
        style={styles.input}
        placeholder="Email"
        value={email}
        onChangeText={setEmail}
        keyboardType="email-address"
        autoCapitalize="none"
      />
      <TextInput
        style={styles.input}
        placeholder="Password"
        value={password}
        onChangeText={setPassword}
        secureTextEntry
      />
      <Button title="Login" onPress={handleLogin} />
      <Button title="Register" onPress={() => navigation.navigate('Register')} />
    </View>
  );
};

const styles = StyleSheet.create({
  container: { flex: 1, justifyContent: 'center', padding: 20 },
  title: { fontSize: 24, marginBottom: 20, textAlign: 'center' },
  input: { height: 40, borderColor: 'gray', borderWidth: 1, marginBottom: 10, paddingHorizontal: 8 },
});

export default LoginScreen;
''',
    ''' import React, { useState } from 'react';
import { View, Text, TextInput, Button, StyleSheet } from 'react-native';
import api from './api';

const RegisterScreen = ({ navigation }) => {
  const [email, setEmail] = useState('');
  const [password, setPassword] = useState('');
  const [username, setUsername] = useState('');

  const handleRegister = async () => {
    try {
      await api.post('/auth/register/', { email, password, username });
      alert('Registration successful! Please verify your email.');
      navigation.navigate('Login');
    } catch (error) {
      console.error(error);
      alert('Registration failed. Please try again.');
    }
  };

  return (
    <View style={styles.container}>
      <Text style={styles.title}>Register</Text>
      <TextInput
        style={styles.input}
        placeholder="Username"
        value={username}
        onChangeText={setUsername}
      />
      <TextInput
        style={styles.input}
        placeholder="Email"
        value={email}
        onChangeText={setEmail}
        keyboardType="email-address"
        autoCapitalize="none"
      />
      <TextInput
        style={styles.input}
        placeholder="Password"
        value={password}
        onChangeText={setPassword}
        secureTextEntry
      />
      <Button title="Register" onPress={handleRegister} />
    </View>
  );
};

const styles = StyleSheet.create({
  container: { flex: 1, justifyContent: 'center', padding: 20 },
  title: { fontSize: 24, marginBottom: 20, textAlign: 'center' },
  input: { height: 40, borderColor: 'gray', borderWidth: 1, marginBottom: 10, paddingHorizontal: 8 },
});

export default RegisterScreen;
''',
    ''' import React, { useEffect, useState } from 'react';
import { View, Text, FlatList, StyleSheet } from 'react-native';
import api from './api';

const HomeScreen = () => {
  const [reports, setReports] = useState([]);

  useEffect(() => {
    const fetchReports = async () => {
      try {
        const response = await api.get('/reports/');
        setReports(response.data);
      } catch (error) {
        console.error(error);
        alert('Error fetching reports.');
      }
    };

    fetchReports();
  }, []);

  return (
    <View style={styles.container}>
      <Text style={styles.title}>Recent Reports</Text>
      <FlatList
        data={reports}
        keyExtractor={(item) => item.id.toString()}
        renderItem={({ item }) => (
          <View style={styles.report}>
            <Text style={styles.reportTitle}>{item.title}</Text>
            <Text>{item.description}</Text>
          </View>
        )}
      />
    </View>
  );
};

const styles = StyleSheet.create({
  container: { flex: 1, padding: 20 },
  title: { fontSize: 24, marginBottom: 20, textAlign: 'center' },
  report: { marginBottom: 15, padding: 10, borderColor: 'gray', borderWidth: 1, borderRadius: 5 },
  reportTitle: { fontWeight: 'bold', marginBottom: 5 },
});

export default HomeScreen;
''',
    ''' import React, { useState } from 'react';
import { View, Text, TextInput, Button, Image, StyleSheet } from 'react-native';
import * as ImagePicker from 'expo-image-picker';
import api from './api';

const FaceScreen = () => {
  const [selectedImage, setSelectedImage] = useState(null);

  const pickImage = async () => {
    const result = await ImagePicker.launchImageLibraryAsync({
      mediaTypes: ImagePicker.MediaTypeOptions.Images,
      allowsEditing: true,
      quality: 1,
    });

    if (!result.canceled) {
      setSelectedImage(result.assets[0].uri);
    }
  };

  const handleUpload = async () => {
    const formData = new FormData();
    formData.append('image', {
      uri: selectedImage,
      name: 'face.jpg',
      type: 'image/jpeg',
    });

    try {
      const response = await api.post('/face-reconstruction/', formData, {
        headers: { 'Content-Type': 'multipart/form-data' },
      });
      alert('Image processed successfully!');
      console.log(response.data);
    } catch (error) {
      console.error(error);
      alert('Error processing the image.');
    }
  };

  return (
    <View style={styles.container}>
      <Text style={styles.title}>Face Reconstruction</Text>
      {selectedImage && <Image source={{ uri: selectedImage }} style={styles.image} />}
      <Button title="Pick an Image" onPress={pickImage} />
      {selectedImage && <Button title="Upload and Process" onPress={handleUpload} />}
    </View>
  );
};

const styles = StyleSheet.create({
  container: { flex: 1, justifyContent: 'center', alignItems: 'center', padding: 20 },
  title: { fontSize: 24, marginBottom: 20, textAlign: 'center' },
  image: { width: 200, height: 200, marginBottom: 10 },
});

export default FaceScreen;
''',
    ''' import React, { useState, useEffect } from 'react';
import { View, Text, TextInput, Button, StyleSheet, Alert, Image } from 'react-native';
import axios from 'axios';
import { useNavigation } from '@react-navigation/native';

const ProfileScreen = ({ route }) => {
  const navigation = useNavigation();
  const { user } = route.params;  // Pass user data via navigation params
  const [profileData, setProfileData] = useState({
    username: user.username,
    email: user.email,
    first_name: user.first_name,
    last_name: user.last_name,
    avatar: user.avatar || '', // If avatar exists
  });
  const [isEditing, setIsEditing] = useState(false);
  const [newAvatar, setNewAvatar] = useState(null);  // Handle avatar update

  // Fetch user profile data from the API
  useEffect(() => {
    const fetchProfileData = async () => {
      try {
        const response = await axios.get('https://127.0.0.1/api/profile/', {
          headers: { Authorization: `Bearer ${user.token}` },
        });
        setProfileData(response.data);
      } catch (error) {
        console.error("Failed to fetch profile data", error);
      }
    };

    fetchProfileData();
  }, [user.token]);

  // Handle profile update
  const handleUpdateProfile = async () => {
    try {
      const formData = new FormData();
      formData.append('username', profileData.username);
      formData.append('email', profileData.email);
      formData.append('first_name', profileData.first_name);
      formData.append('last_name', profileData.last_name);

      // If a new avatar is selected
      if (newAvatar) {
        formData.append('avatar', {
          uri: newAvatar.uri,
          type: newAvatar.type,
          name: newAvatar.name,
        });
      }

      const response = await axios.put('https://127.0.0.1/api/profile/', formData, {
        headers: {
          'Content-Type': 'multipart/form-data',
          Authorization: `Bearer ${user.token}`,
        },
      });

      if (response.status === 200) {
        setIsEditing(false);
        Alert.alert("Profile updated successfully");
      }
    } catch (error) {
      console.error("Failed to update profile", error);
      Alert.alert("Error updating profile");
    }
  };

  return (
    <View style={styles.container}>
      <Text style={styles.header}>Profile</Text>

      <View style={styles.profileInfo}>
        <Text>Username:</Text>
        {isEditing ? (
          <TextInput
            style={styles.input}
            value={profileData.username}
            onChangeText={(text) => setProfileData({ ...profileData, username: text })}
          />
        ) : (
          <Text>{profileData.username}</Text>
        )}
      </View>

      <View style={styles.profileInfo}>
        <Text>Email:</Text>
        {isEditing ? (
          <TextInput
            style={styles.input}
            value={profileData.email}
            onChangeText={(text) => setProfileData({ ...profileData, email: text })}
          />
        ) : (
          <Text>{profileData.email}</Text>
        )}
      </View>

      <View style={styles.profileInfo}>
        <Text>First Name:</Text>
        {isEditing ? (
          <TextInput
            style={styles.input}
            value={profileData.first_name}
            onChangeText={(text) => setProfileData({ ...profileData, first_name: text })}
          />
        ) : (
          <Text>{profileData.first_name}</Text>
        )}
      </View>

      <View style={styles.profileInfo}>
        <Text>Last Name:</Text>
        {isEditing ? (
          <TextInput
            style={styles.input}
            value={profileData.last_name}
            onChangeText={(text) => setProfileData({ ...profileData, last_name: text })}
          />
        ) : (
          <Text>{profileData.last_name}</Text>
        )}
      </View>

      <View style={styles.profileInfo}>
        <Text>Avatar:</Text>
        {profileData.avatar ? (
          <Image source={{ uri: profileData.avatar }} style={styles.avatar} />
        ) : (
          <Text>No Avatar</Text>
        )}
      </View>

      {isEditing && (
        <Button
          title="Choose Avatar"
          onPress={() => {
            // Implement avatar picker here (e.g., using ImagePicker or a similar library)
          }}
        />
      )}

      <Button
        title={isEditing ? "Save Changes" : "Edit Profile"}
        onPress={isEditing ? handleUpdateProfile : () => setIsEditing(true)}
      />
    </View>
  );
};

const styles = StyleSheet.create({
  container: {
    flex: 1,
    padding: 16,
    justifyContent: 'center',
  },
  header: {
    fontSize: 24,
    fontWeight: 'bold',
    marginBottom: 20,
    textAlign: 'center',
  },
  profileInfo: {
    marginBottom: 15,
  },
  input: {
    borderWidth: 1,
    padding: 8,
    marginTop: 5,
    borderRadius: 5,
  },
  avatar: {
    width: 100,
    height: 100,
    borderRadius: 50,
    marginTop: 10,
  },
});

export default ProfileScreen;
''',
    ''' import React, { useEffect, useState } from 'react';
import { View, Text, FlatList, StyleSheet } from 'react-native';
import api from './api';

const ArticleScreen = () => {
  const [articles, setArticles] = useState([]);

  useEffect(() => {
    const fetchArticles = async () => {
      try {
        const response = await api.get('/articles/');
        setArticles(response.data);
      } catch (error) {
        console.error(error);
        alert('Error fetching articles.');
      }
    };

    fetchArticles();
  }, []);

  return (
    <View style={styles.container}>
      <Text style={styles.title}>Articles</Text>
      <FlatList
        data={articles}
        keyExtractor={(item) => item.id.toString()}
        renderItem={({ item }) => (
          <View style={styles.article}>
            <Text style={styles.articleTitle}>{item.title}</Text>
            <Text>{item.content}</Text>
          </View>
        )}
      />
    </View>
  );
};

const styles = StyleSheet.create({
  container: { flex: 1, padding: 20 },
  title: { fontSize: 24, marginBottom: 20, textAlign: 'center' },
  article: { marginBottom: 15, padding: 10, borderColor: 'gray', borderWidth: 1, borderRadius: 5 },
  articleTitle: { fontWeight: 'bold', marginBottom: 5 },
});

export default ArticleScreen;
''',
    ''' import React, { useEffect, useState } from 'react';
import { View, Text, FlatList, StyleSheet } from 'react-native';
import api from './api';

const NotificationScreen = () => {
  const [notifications, setNotifications] = useState([]);

  useEffect(() => {
    const fetchNotifications = async () => {
      try {
        const response = await api.get('/notifications/');
        setNotifications(response.data);
      } catch (error) {
        console.error(error);
        alert('Error fetching notifications.');
      }
    };

    fetchNotifications();
  }, []);

  return (
    <View style={styles.container}>
      <Text style={styles.title}>Notifications</Text>
      <FlatList
        data={notifications}
        keyExtractor={(item) => item.id.toString()}
        renderItem={({ item }) => (
          <View style={styles.notification}>
            <Text style={styles.notificationTitle}>{item.title}</Text>
            <Text>{item.message}</Text>
          </View>
        )}
      />
    </View>
  );
};

const styles = StyleSheet.create({
  container: { flex: 1, padding: 20 },
  title: { fontSize: 24, marginBottom: 20, textAlign: 'center' },
  notification: { marginBottom: 15, padding: 10, borderColor: 'gray', borderWidth: 1, borderRadius: 5 },
  notificationTitle: { fontWeight: 'bold', marginBottom: 5 },
});

export default NotificationScreen;
''',

    ''' import React, { useEffect, useState } from 'react';
import { View, Text, StyleSheet } from 'react-native';
import { BarChart } from 'react-native-chart-kit';
import api from './api';

const GraphicScreen = () => {
  const [graphData, setGraphData] = useState([]);

  useEffect(() => {
    const fetchGraphData = async () => {
      try {
        const response = await api.get('/reports/graph/');
        setGraphData(response.data);
      } catch (error) {
        console.error(error);
        alert('Error fetching graph data.');
      }
    };

    fetchGraphData();
  }, []);

  const data = {
    labels: graphData.map((item) => item.label),
    datasets: [{ data: graphData.map((item) => item.value) }],
  };

  return (
    <View style={styles.container}>
      <Text style={styles.title}>Reports Graph</Text>
      <BarChart
        data={data}
        width={300}
        height={220}
        yAxisLabel=""
        chartConfig={{
          backgroundColor: '#fff',
          backgroundGradientFrom: '#6a51ae',
          backgroundGradientTo: '#6a51ae',
          color: (opacity = 1) => `rgba(255, 255, 255, ${opacity})`,
        }}
        style={styles.graph}
      />
    </View>
  );
};

const styles = StyleSheet.create({
  container: { flex: 1, justifyContent: 'center', alignItems: 'center', padding: 20 },
  title: { fontSize: 24, marginBottom: 20, textAlign: 'center' },
  graph: { marginVertical: 8, borderRadius: 16 },
});

export default GraphicScreen;
''',
    ''' import React, { useState } from 'react';
import { View, Text, TextInput, Button, FlatList, StyleSheet, Image } from 'react-native';
import api from './api';

const SearchScreen = () => {
  const [query, setQuery] = useState('');
  const [results, setResults] = useState([]);
  const [loading, setLoading] = useState(false);

  const handleSearch = async () => {
    setLoading(true);
    try {
      const response = await api.get(`/search/?q=${query}`);
      setResults(response.data);
    } catch (error) {
      console.error(error);
      alert('Error fetching search results.');
    } finally {
      setLoading(false);
    }
  };

  return (
    <View style={styles.container}>
      <TextInput
        style={styles.input}
        placeholder="Search for reports or articles"
        value={query}
        onChangeText={setQuery}
      />
      <Button title="Search" onPress={handleSearch} />
      {loading && <Text>Loading...</Text>}
      <FlatList
        data={results}
        keyExtractor={(item) => item.id.toString()}
        renderItem={({ item }) => (
          <View style={styles.result}>
            <Image source={{ uri: item.image }} style={styles.image} />
            <Text style={styles.title}>{item.title}</Text>
            <Text>{item.description}</Text>
          </View>
        )}
      />
    </View>
  );
};

const styles = StyleSheet.create({
  container: { flex: 1, padding: 20 },
  input: { borderWidth: 1, padding: 10, marginBottom: 10, borderRadius: 5 },
  result: { marginBottom: 20, borderWidth: 1, padding: 10, borderRadius: 5 },
  image: { width: 100, height: 100, marginBottom: 10 },
  title: { fontWeight: 'bold', fontSize: 16, marginBottom: 5 },
});

export default SearchScreen;
''',
    ''' import React from 'react';
import { View, Text, Button, StyleSheet } from 'react-native';

const LandingScreen = ({ navigation }) => {
  return (
    <View style={styles.container}>
      <Text style={styles.title}>Welcome to Missing Person App</Text>
      <Text style={styles.subtitle}>Find or report missing individuals with ease</Text>
      <Button title="Login" onPress={() => navigation.navigate('LoginScreen')} />
      <Button title="Register" onPress={() => navigation.navigate('RegisterScreen')} />
    </View>
  );
};

const styles = StyleSheet.create({
  container: { flex: 1, justifyContent: 'center', alignItems: 'center', padding: 20 },
  title: { fontSize: 24, fontWeight: 'bold', marginBottom: 10 },
  subtitle: { fontSize: 16, color: 'gray', marginBottom: 20 },
});

export default LandingScreen;
''',
    ''' import React, { useState } from 'react';
import { View, Text, TextInput, Button, StyleSheet } from 'react-native';
import api from './api';

const VerificationEmailScreen = ({ navigation }) => {
  const [code, setCode] = useState('');

  const handleVerification = async () => {
    try {
      const response = await api.post('/verify-email/', { code });
      alert('Email verified successfully!');
      navigation.navigate('LoginScreen');
    } catch (error) {
      console.error(error);
      alert('Invalid verification code.');
    }
  };

  return (
    <View style={styles.container}>
      <Text style={styles.title}>Verify Your Email</Text>
      <TextInput
        style={styles.input}
        placeholder="Enter verification code"
        value={code}
        onChangeText={setCode}
      />
      <Button title="Verify" onPress={handleVerification} />
    </View>
  );
};

const styles = StyleSheet.create({
  container: { flex: 1, justifyContent: 'center', alignItems: 'center', padding: 20 },
  title: { fontSize: 24, marginBottom: 20 },
  input: { borderWidth: 1, padding: 10, marginBottom: 20, borderRadius: 5 },
});

export default VerificationEmailScreen;
''',
    ''' import React, { useState } from 'react';
import { View, Text, TextInput, Button, StyleSheet } from 'react-native';
import api from './api';

const ForgotPassword = ({ navigation }) => {
  const [email, setEmail] = useState('');

  const handleForgotPassword = async () => {
    try {
      await api.post('/forgot-password/', { email });
      alert('Password reset link sent to your email!');
      navigation.navigate('LoginScreen');
    } catch (error) {
      console.error(error);
      alert('Error sending reset link.');
    }
  };

  return (
    <View style={styles.container}>
      <Text style={styles.title}>Forgot Password</Text>
      <TextInput
        style={styles.input}
        placeholder="Enter your email"
        value={email}
        onChangeText={setEmail}
      />
      <Button title="Send Reset Link" onPress={handleForgotPassword} />
    </View>
  );
};

const styles = StyleSheet.create({
  container: { flex: 1, justifyContent: 'center', alignItems: 'center', padding: 20 },
  title: { fontSize: 24, marginBottom: 20 },
  input: { borderWidth: 1, padding: 10, marginBottom: 20, borderRadius: 5 },
});

export default ForgotPassword;
''',
    ''' import React, { useState } from 'react';
import { View, Text, TextInput, Button, StyleSheet } from 'react-native';
import api from './api';

const LaporanPelaporan = () => {
  const [title, setTitle] = useState('');
  const [description, setDescription] = useState('');

  const handleSubmit = async () => {
    try {
      await api.post('/reports/', { title, description });
      alert('Report submitted successfully!');
    } catch (error) {
      console.error(error);
      alert('Error submitting report.');
    }
  };

  return (
    <View style={styles.container}>
      <Text style={styles.title}>Create a Report</Text>
      <TextInput
        style={styles.input}
        placeholder="Report Title"
        value={title}
        onChangeText={setTitle}
      />
      <TextInput
        style={styles.input}
        placeholder="Description"
        value={description}
        onChangeText={setDescription}
        multiline
      />
      <Button title="Submit Report" onPress={handleSubmit} />
    </View>
  );
};

const styles = StyleSheet.create({
  container: { flex: 1, padding: 20 },
  title: { fontSize: 24, marginBottom: 20 },
  input: { borderWidth: 1, padding: 10, marginBottom: 10, borderRadius: 5 },
});

export default LaporanPelaporan;
'''

]

for file_name, content in zip(file_names, file_contents):
    file_path = os.path.join(folder_name, file_name)
    with open(file_path, "w") as file:
        file.write(content)

print(f"{len(file_names)} berhasil '{folder_name}'.")


13 berhasil 'screens'.


In [26]:
import os
import shutil

folders_to_manage = [
    "/content/drive/MyDrive/FaceReconstruction/frontend/frontend/assets/images",
    "/content/drive/MyDrive/FaceReconstruction/frontend/frontend/assets/fonts",
]

def create_folders(folder_list):
    for folder in folder_list:
        try:
            os.makedirs(folder, exist_ok=True)
            print(f"Folder created: {folder}")
        except Exception as e:
            print(f"Error creating folder {folder}: {e}")

print("Creating Folders")
create_folders(folders_to_manage)

Creating Folders
Folder created: /content/drive/MyDrive/FaceReconstruction/frontend/frontend/assets/images
Folder created: /content/drive/MyDrive/FaceReconstruction/frontend/frontend/assets/fonts


In [27]:
%cd /content/drive/MyDrive/FaceReconstruction/frontend/frontend

/content/drive/MyDrive/FaceReconstruction/frontend/frontend


In [28]:
!npx expo start

⠙Starting project at /content/drive/MyDrive/FaceReconstruction/frontend/frontend
Error: Cannot find module 'ajv'
Require stack:
- /content/drive/MyDrive/FaceReconstruction/frontend/frontend/node_modules/schema-utils/dist/validate.js
- /content/drive/MyDrive/FaceReconstruction/frontend/frontend/node_modules/schema-utils/dist/index.js
- /content/drive/MyDrive/FaceReconstruction/frontend/frontend/node_modules/expo-router/plugin/build/index.js
- /content/drive/MyDrive/FaceReconstruction/frontend/frontend/node_modules/expo-router/app.plugin.js
- /content/drive/MyDrive/FaceReconstruction/frontend/frontend/node_modules/@expo/config/node_modules/@expo/config-plugins/build/utils/plugin-resolver.js
- /content/drive/MyDrive/FaceReconstruction/frontend/frontend/node_modules/@expo/config/node_modules/@expo/config-plugins/build/plugins/withStaticPlugin.js
- /content/drive/MyDrive/FaceReconstruction/frontend/frontend/node_modules/@expo/config/node_modules/@expo/config-plugins/build/plugins/withPlugin